In [1]:
import findspark
findspark.init("C:\\Users\\Software\\spark-3.3.1-bin-hadoop3\\bin\\..")

In [2]:
from pyspark import SparkContext

In [3]:
sc = SparkContext("local","Broadcast and Accumulator apps")

In [4]:
palabras_nuevo = sc.broadcast(["scala","java","hadoop","spark","akka"])

In [5]:
data = palabras_nuevo.value

In [6]:
print("Datos almacenados->%s"% (data))

Datos almacenados->['scala', 'java', 'hadoop', 'spark', 'akka']


In [7]:
elemento = palabras_nuevo.value[2]
print("Imprimiendo un elemento particular en RDD-> %s"% (elemento))

Imprimiendo un elemento particular en RDD-> hadoop


In [8]:
#En el siguiente ejemplo se definen los estados USA enun variable Map 
#y se distribuye tal variable utilizando broadcast() y luego se
#utilizan estas variables en una transformación map() de RDD
estados= {"NY":"New York", "CA":"California", "FL":"Florida"}

In [9]:
broadcastEstados = sc.broadcast(estados)

In [10]:
data = [("James","Smith","USA","CA"),
    ("Michael","Rose","USA","NY"),
    ("Robert","Williams","USA","CA"),
    ("Maria","Jones","USA","FL")
  ]

In [11]:
rdd= sc.parallelize(data)

In [12]:
def conversor_estado(codigo):
    return broadcastEstados.value[codigo]

In [121]:
print("Los nombres de los estados es: %s" % broadcastEstados.value)
print("EL nombre del estado es: %s" % estados["NY"])
print("EL nombre del estado es: %s" % broadcastEstados.value["NY"])

Los nombres de los estados es: {'NY': 'New York', 'CA': 'California', 'FL': 'Florida'}
EL nombre del estado es: New York
EL nombre del estado es: New York


In [13]:
resultado = rdd.map(lambda x:(x[0],x[1],x[2],conversor_estado(x[3]))).collect()

In [14]:
print(resultado)

[('James', 'Smith', 'USA', 'California'), ('Michael', 'Rose', 'USA', 'New York'), ('Robert', 'Williams', 'USA', 'California'), ('Maria', 'Jones', 'USA', 'Florida')]


In [15]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

In [33]:
states= {"NY":"New York", "CA":"California", "FL":"Florida"}
broadcastStates = spark.sparkContext.broadcast(states)
columns = ["firstname","lastname","country","state"]
df = spark.createDataFrame(data=data, schema = columns)
#df1=df.where((df["state"].isin(broadcastStates.value)))
print (df["state"])
df.printSchema()
df.show(truncate=False)
print (broadcastStates.value)


Column<'state'>
root
 |-- firstname: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- country: string (nullable = true)
 |-- state: string (nullable = true)

+---------+--------+-------+-----+
|firstname|lastname|country|state|
+---------+--------+-------+-----+
|James    |Smith   |USA    |CA   |
|Michael  |Rose    |USA    |NY   |
|Robert   |Williams|USA    |CA   |
|Maria    |Jones   |USA    |FL   |
+---------+--------+-------+-----+

{'NY': 'New York', 'CA': 'California', 'FL': 'Florida'}


In [106]:
#Una variable Acumulador tiene un atributo llamado 'value' que es 
#similar al que tiene una variable Broadcast.
#Almacena los datos y se utiliza para devolver el valor del 
#acumulador, pero sólo es utilizable en el "driver"
num = sc.accumulator(10)

In [107]:
def f(x):
    global num
    num+=x
    #return num

In [108]:
rdd = sc.parallelize([20,30,40,50])

In [109]:
rdd=rdd.map(lambda x: f(x))
for elemento in rdd.collect():
    print(elemento)

None
None
None
None


In [110]:
print("El valor acumulados es-> %i"%num.value)

El valor acumulados es-> 150
